In [ ]:
# installations
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
# read the data file into df
df = pd.read_csv("Data files/mmsa-icu-beds.csv")
print(df.head(10))
print(df.columns)

1. Which 10 regions have the highest percentage of at-risk population?

In [ ]:
df_sorted = df.copy()
Regions_at_risk=df_sorted.sort_values(by='total_percent_at_risk',ascending=False).head(10)
print(Regions_at_risk)

In [ ]:
Regions_at_risk=df_sorted.sort_values(by='total_percent_at_risk',ascending=False)
top10=Regions_at_risk.iloc[:10,:]
print(top10)


In [ ]:
Regions_at_risk=df_sorted.sort_values(by='total_percent_at_risk',ascending=False).reset_index(drop=True)
top10=Regions_at_risk.loc[:9,:]
print(top10)

2. Which 10 areas have the fewest ICU beds per 1,000 high-risk people?

In [ ]:
df_sorted = df.copy()
df_sorted['icu_beds_pre_1000']=(df_sorted['icu_beds'] / df_sorted['total_at_risk'])*1000
sorted_value=df_sorted.sort_values(by='icu_beds_pre_1000',ascending=True).head(10)
print(sorted_value)

In [ ]:
sorted_value=df_sorted.sort_values(by='icu_beds_pre_1000',ascending=True).reset_index(drop=True)
top10=sorted_value.loc[:9,:]
print(top10)

In [ ]:
sorted_value=df_sorted.sort_values(by='icu_beds_pre_1000',ascending=True)
top10=sorted_value.iloc[:10,:]
print(top10)

3. Which 10 hospitals are facing the greatest burden in terms of high-risk patients per hospital?

In [ ]:
df_sorted = df.copy()
top_10_burdened_hospitals = df_sorted.sort_values(by='high_risk_per_hospital', ascending=False).head(10)
print(top_10_burdened_hospitals)

In [ ]:
burdened_hospitals = df_sorted.sort_values(by='high_risk_per_hospital', ascending=False)
top_10=burdened_hospitals.iloc[:10,:]
print(top_10)

In [ ]:
burdened_hospitals = df_sorted.sort_values(by='high_risk_per_hospital', ascending=False).reset_index(drop=True)
top_10=burdened_hospitals.loc[:9,:]
print(top_10)

4.Filter areas with >60% at-risk population and <30 ICU beds.

In [ ]:
df_sorted = df.copy()
data = df_sorted[(df_sorted['total_percent_at_risk'] > '60%' ) & (df_sorted['icu_beds']<30)]
print(data)

5. What is the average at-risk percentage by state?

In [ ]:
df_sorted = df.copy()
#df_sorted['total_percent_at_risk'] = df_sorted['total_percent_at_risk'].str.replace('%', '', regex=False)
#df_sorted['total_percent_at_risk'] = pd.to_numeric(df_sorted['total_percent_at_risk'], errors='coerce')
df_sorted['state'] = df_sorted['MMSA'].str.split(',').str[-1].str.strip()
avg_risk_by_state = df_sorted.groupby('state')['total_percent_at_risk'].mean().reset_index()
print(avg_risk_by_state)

6. Is there a correlation between ICU beds and the % at-risk?

In [ ]:
df_sorted = df.copy()
df_clean = df_sorted.dropna(subset=['icu_beds', 'total_percent_at_risk'])
correlation = df_clean['icu_beds'].corr(df_clean['total_percent_at_risk'])
print(f"Correlation between ICU beds and % at-risk: {correlation:.4f}")

7. Visualize total percent at-risk vs. high-risk per ICU bed.

In [ ]:
df_sorted = df.copy()
# Clean and convert data
df_sorted['total_percent_at_risk'] = df_sorted['total_percent_at_risk'].astype(str).str.replace('%', '', regex=False)
df_sorted['total_percent_at_risk'] = pd.to_numeric(df_sorted['total_percent_at_risk'], errors='coerce')
df_sorted['high_risk_per_ICU_bed'] = pd.to_numeric(df_sorted['high_risk_per_ICU_bed'], errors='coerce')

# Drop missing values
df_clean = df_sorted.dropna(subset=['total_percent_at_risk', 'high_risk_per_ICU_bed'])

# Create the scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_clean,
    x='total_percent_at_risk',
    y='high_risk_per_ICU_bed',
    color='blue',
    edgecolor='white'
)

plt.title('Total % At-Risk vs. High-Risk Patients per ICU Bed')
plt.xlabel('Total Percent At-Risk')
plt.ylabel('High-Risk Patients per ICU Bed')
plt.show()

8. Estimate ICU shortfall if we need 1 ICU bed per 500 high-risk individuals.


In [ ]:
df_sorted = df.copy()
# Calculate required ICU beds
df_sorted['required_icu_beds'] = df_sorted['total_at_risk'] / 500

# Estimate shortfall
df_sorted['icu_shortfall'] = df_sorted['required_icu_beds'] - df_sorted['icu_beds']

# Optional: filter areas with an actual shortfall
icu_shortage_df = df_sorted[df_sorted['icu_shortfall'] > 0]

# Display or sort the top shortfalls
icu_shortage_df = icu_shortage_df.sort_values(by='icu_shortfall', ascending=False)

# Show key columns
print(icu_shortage_df[['MMSA', 'total_at_risk', 'icu_beds', 'required_icu_beds', 'icu_shortfall']].head(10))

9. Rank regions by high-risk individuals per hospital.

In [ ]:
df_sorted = df.copy()
# Sort the DataFrame in descending order
ranked_df = df_sorted.sort_values(by='high_risk_per_hospital', ascending=False)

ranked_df['rank'] = ranked_df['high_risk_per_hospital'].rank(method='min', ascending=False)

# Reset index for neatness
ranked_df = ranked_df.reset_index(drop=True)

# View top 10 regions
print(ranked_df[['MMSA', 'high_risk_per_hospital', 'rank']].head(10))



10. Create a composite risk score to rank all regions.

In [ ]:
df_sorted = df.copy()

from sklearn.preprocessing import MinMaxScaler

cols_to_normalize = [
    'total_percent_at_risk',
    'high_risk_per_icu_bed',
    'high_risk_per_hospital',
    'icu_beds',
    'hospitals'
]

scaler = MinMaxScaler()
df_normalized = df.copy()
df_normalized[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])


df_normalized['composite_risk_score'] = (
    df_normalized['total_percent_at_risk'] +
    df_normalized['high_risk_per_icu_bed'] +
    df_normalized['high_risk_per_hospital'] -
    df_normalized['icu_beds'] -
    df_normalized['hospitals']
)

df_normalized['risk_rank'] = df_normalized['composite_risk_score'].rank(ascending=False)


11. Top 10 Regions with Most ICU Beds per Capita

In [ ]:
df_sorted = df.copy()
df_sorted['icu_beds_per_100k_at_risk'] = (df_sorted['icu_beds'] / df_sorted['total_at_risk']) 

top_icu_per_capita = df_sorted.sort_values(by='icu_beds_per_100k_at_risk', ascending=False).head(10)

print(top_icu_per_capita[['MMSA', 'icu_beds', 'total_at_risk', 'icu_beds_per_100k_at_risk']])

12. Regions With No ICU Beds,


In [ ]:
df_sorted = df.copy()
no_icu_regions = df_sorted[df_sorted['icu_beds'] == 0]
print(no_icu_regions[['MMSA', 'total_at_risk', 'hospitals']])


    13. Regions With No Hospitals


In [ ]:
df_sorted = df.copy()
no_hospital_regions = df_sorted[df_sorted['hospitals'] == 0]
print(no_hospital_regions[['MMSA', 'total_at_risk', 'icu_beds']])


    14. Compare ICU Pressure Between Two Areas


15. Percent of High-Risk Individuals vs Hospital Count (Correlation)


In [ ]:
df_sorted = df.copy()

df_sorted['total_percent_at_risk'] = df_sorted['total_percent_at_risk'].str.replace('%', '', regex=False)
df_sorted['total_percent_at_risk'] = pd.to_numeric(df_sorted['total_percent_at_risk'], errors='coerce')
df_clean = df_sorted[['total_percent_at_risk', 'hospitals']].dropna()

# Compute Pearson correlation
correlation = df_clean['total_percent_at_risk'].corr(df_clean['hospitals'])

print(f"Correlation between percent at risk and hospital count: {correlation:.2f}")

16. Bucket Regions by Risk Level


In [ ]:
df_sorted = df.copy()

df_sorted['total_percent_at_risk'] = df_sorted['total_percent_at_risk'].str.replace('%', '', regex=False)
df_sorted['total_percent_at_risk'] = pd.to_numeric(df_sorted['total_percent_at_risk'], errors='coerce')

# Function to bucket by risk level
def bucket_risk_level(percent):
    if percent <= 40:
        return 'Low'
    elif percent <= 55:
        return 'Moderate'
    elif percent <= 65:
        return 'High'
    else:
        return 'Very High'

# Apply bucket
df_sorted['risk_bucket'] = df_sorted['total_percent_at_risk'].apply(bucket_risk_level)

# Optional: view bucketed data
print(df_sorted[['MMSA', 'total_percent_at_risk', 'risk_bucket']])

17. Top Regions Needing Emergency Resource Allocation


In [95]:
df_sorted = df.copy()

df_sorted['icu_beds'] = df_sorted['icu_beds'].fillna(0)
df_sorted['hospitals'] = df_sorted['hospitals'].fillna(0)

# Assign scores (higher = more urgent need)
df_sorted['risk_score'] = df_sorted['total_percent_at_risk']
df_sorted['icu_burden_score'] = df_sorted['high_risk_per_ICU_bed'].replace(np.nan, 1e6)  # NA = very high burden
df_sorted['hospital_burden_score'] = df_sorted['high_risk_per_hospital'].replace(np.nan, 1e6)

df_sorted['composite_need_score'] = (
    df_sorted['risk_score'].rank(ascending=False) +
    df_sorted['icu_burden_score'].rank(ascending=False) +
    df_sorted['hospital_burden_score'].rank(ascending=False)
)

# Sort by highest need
top_regions = df_sorted.sort_values('composite_need_score', ascending=False).head(5)
print(top_regions[['MMSA', 'total_percent_at_risk', 'icu_beds', 'hospitals', 'composite_need_score']])


                   MMSA total_percent_at_risk  icu_beds  hospitals  \
135       Rochester, MN                54.72%     218.0        4.0   
112      Provo-Orem, UT                38.92%     122.0        6.0   
106  Grand Forks, ND-MN                54.83%      31.0        4.0   
114         Lincoln, NE                53.81%     108.0        5.0   
132       Duluth, MN-WI                60.34%     185.0       11.0   

     composite_need_score  
135                 365.0  
112                 359.0  
106                 351.0  
114                 339.0  
132                 333.0  


18. Calculate ICU Utilization Ratio (Projected) 


19. Histogram of At-Risk Percentages 


20. State-Wise ICU Capacity Ranking